In [1]:
from utils.data_preparation import *

In [2]:
from model.hierarchical_size_model_new import *
from model.hierarchical_model_return_status import *

class HierarchicalFullModel:
    def __init__(self, size_model, return_status_model):
        self.size_model = size_model
        self.return_status_model = return_status_model

    def predict(self, test):
        return_status_result = self.return_status_model.predict(test)
        for result_description, result_label in [ ("fit", FIT_LABEL), ("large", LARGE_LABEL), ("small", SMALL_LABEL)]:
            all_same = test.copy()
            all_same["result"] = result_label
            all_same_result = self.size_model.predict(all_same)
            return_status_result[f"predicted_prob_{result_description}"] = all_same_result["predicted_prob"]
            return_status_result[f"predicted_size_{result_description}"] = all_same_result["predicted_size"]
            return_status_result[f"all_sizes_results_{result_description}"] = all_same_result["all_sizes_results"]
            return_status_result[f"predicted_full_prob_{result_description}"] = return_status_result[result_label] * return_status_result[f"predicted_prob_{result_description}"]
        full_prob_columns = [f"predicted_full_prob_{result_desc}" for result_desc in ["fit", "small", "large"]]
        full_prob_columns_to_return_status_desc = {f"predicted_full_prob_{result_desc}":result_desc for result_desc in ["fit", "small", "large"]}
        return_status_result["predicted_pair_prob"] = return_status_result[full_prob_columns].max(axis=1)
        return_status_result["predicted_pair_return_status"] = return_status_result[full_prob_columns].idxmax(axis=1).replace(full_prob_columns_to_return_status_desc)
        return_status_result["predicted_pair_size"] = return_status_result.apply(lambda row: row[f"predicted_size_{row['predicted_pair_return_status']}"], axis=1)
        return return_status_result

    def get_target_prob(self, test):
        return_status_result = self.return_status_model.predict(test)
        full_result = self.size_model.size_prob(return_status_result)
        full_result["target_prob"] = full_result["size_prob"] * full_result["return_status_prob"]
        return full_result

In [3]:
test = get_data_from_config({"dataset_type":"test", "test_size": 0.025})
full = get_processed_renttherunway_data()

In [4]:
size_model = HierarchicalSize(full)
return_status_model = HierarchicalStatus(df=full)
full_model = HierarchicalFullModel(size_model, return_status_model)

creating parameters
creating constant parameters
Initializing model... Model initiated
Trainset prepared


In [5]:
full_result = full_model.predict(test)

In [7]:
full_result[full_result["predicted_pair_return_status"]!="fit"]

,result_original,user_id_original,item_id_original,size,review_date,category,user_id,item_id,result,0_by_article,...,predicted_size_large,all_sizes_results_large,predicted_full_prob_large,predicted_prob_small,predicted_size_small,all_sizes_results_small,predicted_full_prob_small,predicted_pair_prob,predicted_pair_return_status,predicted_pair_size
1120,large,174995,2610728,14,"December 9, 2016",pullover,18397,5125,1,4.0,...,16,"[3.4382901484838185e-05, 5.174775601360098e-05...",0.079821,0.163922,14,"[8.018042366582666e-05, 0.0001098383102034076,...",0.002915,0.079821,large,16
1850,large,841772,2622679,8,"November 7, 2016",coat,88825,5149,1,3.0,...,10,"[2.2294471456019024e-05, 0.0001521515583798429...",0.073270,0.158205,7,"[0.0022868688960731224, 0.00459903368537325, 0...",0.004216,0.073270,large,10
2727,large,317664,2521310,8,"November 10, 2016",cardigan,33528,4942,1,14.0,...,8,"[1.9139036127563636e-06, 1.4004699437778788e-0...",0.094187,0.154620,7,"[0.0031762090787881172, 0.005986225851452234, ...",0.000585,0.094187,large,8
4029,large,125457,2767494,14,"August 31, 2016",hoodie,13228,5441,1,2.0,...,13,"[0.001536707872989786, 0.0014879999405861437, ...",0.072378,0.114493,10,"[0.0008203083048394004, 0.0035829000034004474,...",0.000914,0.072378,large,13
4252,large,3696,2549740,20,"October 20, 2015",culottes,410,4998,1,5.0,...,19,"[3.6212812694034734e-07, 6.52895275229624e-07,...",0.059187,0.125903,18,"[1.3655252810321913e-06, 2.417694302867871e-06...",0.005790,0.059187,large,19


In [10]:
full_result_just_target_prob = full_model.get_target_prob(test)
np.log(full_result_just_target_prob["target_prob"]).replace({-np.inf:np.nan}).mean()

/home/martyna/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


-2.839636992312736